In [13]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.notebook import trange, tqdm, tqdm_notebook
from joblib import Parallel, delayed
import re
import h5py
import tifffile as tiff


In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
data_processed = data_dir / 'processed'
data_raw = r'Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\Sun Oh experiments\IF intensity on fixed permeabilized cells 27Mar24'


In [16]:
import os
 
os.environ['JDK_HOME'] = "C:\Program Files\Java\jdk-17"
os.environ['JAVA_HOME'] = "C:\Program Files\Java\jdk-17"

os.environ['PATH'] += ';C:\\Program Files\\Java\\jdk-17\\jre\\bin\\server\\;C:\\Program Files\\Java\\jdk-17\\bin\\;C:\\Program Files\\Java\\jdk-17\\bin\\jar.exe'

# Get info

In [20]:
markers_map = {
    'cycle1': {
        1: 'Hoechst', 
        2: 'DR5',
        3: 'TRAIL',
    },
    'cycle2': {
        1: 'Hoechst', 
        3: 'Phalloidin',
    },
}

def get_info(data_raw, marker_dict):
    cells = []
    conditions = []
    types = []

    cycles = []
    channels = []
    markers = []
    rois = []
    z_stacks = []
    paths = [] 
    
    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        if 'after nuclease' in dirpath or 'Test' in dirpath or 'wrong' in dirpath:
            continue
        
        for name in sorted(filenames):
            if "tif" in name and "sti" not in name \
            and 'overlay' not in name.lower() \
            and 'Composite' not in name:
                # Get information from image name
                d_split = dirpath.split('\\')

                cell = d_split[-2]
                condition = d_split[-1].split(' ')[1]
                t = 'Fixed'

                n_split = name.split('_')
                ch = int(n_split[-1][-5])
                roi = int(n_split[1])
                z = int(n_split[2][1:])
                if 'Phall' in dirpath:
                    cycle = 'cycle2'
                else:
                    cycle = 'cycle1'
                try: marker = marker_dict[cycle][ch] 
                except: continue

                cells.append(cell)
                conditions.append(condition)
                types.append(t)

                cycles.append(cycle)
                channels.append(ch)
                markers.append(marker)
                rois.append(roi)
                z_stacks.append(z)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            "Cell": cells,
            "Condition": conditions,
            'FOV': types, 
            "Cycle": cycles,
            "Channels": channels,
            "Markers": markers,
            "ROI": rois,
            "Z": z_stacks,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df

In [21]:
df_meta_path = data_dir / 'sun_IF_3D' / 'metadata' / 'info.csv'
df_meta_path.parent.mkdir(parents=True, exist_ok=True)

df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw, markers_map)
    # df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Created df


In [22]:
df.head()

,Cell,Condition,FOV,Cycle,Channels,Markers,ROI,Z,Path
0,HCC827,ctrl,Fixed,cycle1,1,Hoechst,1,1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
1,HCC827,ctrl,Fixed,cycle1,2,DR5,1,1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
2,HCC827,ctrl,Fixed,cycle1,3,TRAIL,1,1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
3,HCC827,ctrl,Fixed,cycle1,1,Hoechst,1,2,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
4,HCC827,ctrl,Fixed,cycle1,2,DR5,1,2,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."


In [24]:
df.groupby(['Cell', 'Condition', 'FOV', 'ROI']).size()

Cell    Condition  FOV    ROI
HCC827  O          Fixed  1      134
                          2      134
                          3      134
                          4      134
                          5      134
                          6      134
                          7      134
                          8      134
                          9      134
        ctrl       Fixed  1      123
                          2      123
                          3      123
                          4      123
                          5      123
                          6      123
                          7      123
                          8      123
                          9      123
PC9     O          Fixed  1      129
                          2      129
                          3      129
                          4      129
                          5      129
                          6      129
                          7      129
                          8      129
        

# Save HDF5

In [30]:
import h5py
from natsort import natsort_keygen, natsorted

def save_hdf5(
    path: str, name: str, data: np.ndarray, attr_dict=None, mode: str = "a"
) -> None:
    # Read h5 file
    hf = h5py.File(path, mode)
    # Create z_stack_dataset
    if hf.get(name) is None:
        data_shape = data.shape
        data_type = data.dtype
        max_shape = (data_shape[0],) + data_shape[1:]
        dset = hf.create_dataset(
            name,
            shape=data_shape,
            maxshape=max_shape,
            chunks=True,
            dtype=data_type,
            compression="gzip",
        )
        dset[:] = data
        if attr_dict is not None:
            for attr_key, attr_val in attr_dict.items():
                dset.attrs[attr_key] = attr_val
    else:
        print(f"Dataset {name} exists")

    hf.close()

def test_data_exist(file_path, name):
    with h5py.File(file_path, "r") as h5fout:
        if name in h5fout:
            return True
        else:
            return False
        
def read_img(path):
    return skimage.io.imread(path, as_gray=True)

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

In [31]:
df

,Cell,Condition,FOV,Cycle,Channels,Markers,ROI,Z,Path
0,HCC827,ctrl,Fixed,cycle1,1,Hoechst,1,1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
1,HCC827,ctrl,Fixed,cycle1,2,DR5,1,1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
2,HCC827,ctrl,Fixed,cycle1,3,TRAIL,1,1,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
3,HCC827,ctrl,Fixed,cycle1,1,Hoechst,1,2,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
4,HCC827,ctrl,Fixed,cycle1,2,DR5,1,2,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
...,...,...,...,...,...,...,...,...,...
4567,PC9,O,Fixed,cycle2,3,Phalloidin,9,22,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
4568,PC9,O,Fixed,cycle2,1,Hoechst,9,23,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
4569,PC9,O,Fixed,cycle2,3,Phalloidin,9,23,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."
4570,PC9,O,Fixed,cycle2,1,Hoechst,9,24,"Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\S..."


In [32]:
df_imgs_path = data_dir / 'sun_IF_3D' / 'metadata' / 'imgs.csv'
df_imgs_path.parent.mkdir(parents=True, exist_ok=True)

temp_path = data_dir  /'sun_IF_3D' /  'hdf5' / 'raw'
temp_path.mkdir(parents=True, exist_ok=True)

df_exist = df_imgs_path.is_file()

if not df_exist:
    print('Created df')
    
    group = df.groupby(['Cell', 'Condition', 'FOV', 'ROI'])
    rows = []

    for name, df_group in tqdm(group, total=len(group)):
        df_group = df_group.sort_values( # Sort by cycle and channels
            by=["Cycle", "Channels", 'Z'],
            key=natsort_keygen()
        )
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])
        
        group_channel = df_group.groupby(['Cycle', 'Channels'])
        for n, df_channel in group_channel:
            try:
                if test_data_exist(file_path, '_'.join(np.array(n).astype(str))):
                    continue
            except:pass
            
            marker = df_channel.iloc[0].Markers
            paths = df_channel.Path.to_numpy()

            imgs = joblib_loop(read_img, paths)
            imgs = np.array(imgs)
            info = {"Cycle": n[0], "Channel": n[1], "Marker": marker, "Z": df_channel.Z.to_numpy()}
            
            # hdf5 as Channel -> Z mapping
            save_hdf5(file_path, '_'.join(np.array(n).astype(str)), imgs, info)
    df_imgs = pd.DataFrame(rows, columns=['Cell', 'Condition','FOV', 'ROI', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Created df


  0%|          | 0/36 [00:00<?, ?it/s]

In [33]:
df_imgs

,Cell,Condition,FOV,ROI,Path
0,HCC827,O,Fixed,1,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
1,HCC827,O,Fixed,2,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
2,HCC827,O,Fixed,3,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
3,HCC827,O,Fixed,4,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
4,HCC827,O,Fixed,5,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
5,HCC827,O,Fixed,6,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
6,HCC827,O,Fixed,7,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
7,HCC827,O,Fixed,8,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
8,HCC827,O,Fixed,9,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
9,HCC827,ctrl,Fixed,1,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...


# Calculate best focus

The best focus for each ROI in one FOV is varying because of the lens heating up. Therefore, we need to extract the individual z stack best focus

In [36]:
from joblib import Parallel, delayed
import cv2 
import scipy 
from collections import defaultdict 

# compute laplacian for each z plane in parallel
def computeLapVar(plane):
    
    var = cv2.Laplacian(plane, cv2.CV_64F, ksize = 31)
    var = np.var(var)
    
    return var

# find focus plane via Laplacian variance
def findFocusLapVar(subStack):
    
    lapVar = Parallel(n_jobs = -1, prefer = 'threads', verbose = 0)(delayed(computeLapVar)(subStack[ii, :, :]) for ii in range(subStack.shape[0]))
    idxFocus = np.argmax(lapVar)

    xRange = np.arange(0, len(lapVar))
    
    # compute steepest gradient in variance to find focus plane
    grad = np.gradient(lapVar)
    grad = np.square(grad)
    
    # extract peaks of gradient
    thresh = np.percentile(grad, 50)
    # peaks with min horizontal distance
    peaks, props = scipy.signal.find_peaks(x = grad, height = thresh, distance = 1)
    heights = props['peak_heights']
    
    # idxFocus = np.argmax(grad) + 2
    if len(peaks) == 0:
        idxFocus = len(lapVar) // 2 # middle
        
    else:
        idxFocus = peaks[0] # tallest peak
        
    if idxFocus > len(lapVar) - 2: # exceeds length, out of bounds
        idxFocus = len(lapVar) - 2
        
    return idxFocus, peaks, lapVar

# register cycles across Z by matching focus planes
def registerAlongZ(fixed, moving): # each ZYX
     
    # match moving to fixed
    focusFixed, peaksFixed, lapVarFixed= findFocusLapVar(fixed) # focus plane of fixed
    focusMoving, peaksMoving, lapVarMoving = findFocusLapVar(moving) # focus plane of moving
    
    # if multiple peaks found, choose closest to middle 
    focusFixed = np.argmax(lapVarFixed)
        
    focusMoving = np.argmax(lapVarMoving)
    
    # print('Fixed focus plane is', focusFixed + 1, '/', fixed.shape[0])
    # print('Moving focus plane is', focusMoving + 1, '/', moving.shape[0])
    shift = focusFixed - focusMoving # z shift applied to moving to match fixed
    
    return shift

In [37]:
df_imgs_path = data_dir / 'sun_IF_3D' / 'metadata' / 'imgs.csv'
df_imgs = pd.read_csv(df_imgs_path)
df_imgs.head()

,Cell,Condition,FOV,ROI,Path
0,HCC827,O,Fixed,1,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
1,HCC827,O,Fixed,2,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
2,HCC827,O,Fixed,3,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
3,HCC827,O,Fixed,4,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
4,HCC827,O,Fixed,5,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...


In [38]:
from collections import defaultdict
# Define fov to compute the z shifts one
z_shifts_all = defaultdict(dict)
group = df_imgs.groupby(['Cell', 'Condition', 'FOV'])

for name, df_group in group:
    # Read imgae and plot one z
    for row in df_group.itertuples():
        path = row.Path
        with h5py.File(path, "r") as f:
            for k in f.keys():
                imgs = f[k][:]
                break

        # Perform z shifts analysis
        z_shifts = defaultdict(list)
        with h5py.File(path, "r") as f:
            for k in tqdm(natsorted(f.keys()), total=len(f.keys()), leave=False):
                cycle = k.split('_')[0]
                channel = f[k].attrs['Channel']

                if channel == 1:
                    imgs = f[k][:]

                    if cycle == 'cycle1':
                        fixed = imgs
                        z_shifts['cycle1'] = 0
                        continue
                    else:
                        moving = imgs
                    shift = registerAlongZ(fixed, moving)
                    
                    z_shifts[cycle] = shift
        z_shifts_all['_'.join(name)][row.ROI] = z_shifts

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [39]:
z_shifts_all

defaultdict(dict,
            {'HCC827_O_Fixed': {1: defaultdict(list,
                          {'cycle1': 0, 'cycle2': 3}),
              2: defaultdict(list, {'cycle1': 0, 'cycle2': 5}),
              3: defaultdict(list, {'cycle1': 0, 'cycle2': 5}),
              4: defaultdict(list, {'cycle1': 0, 'cycle2': 5}),
              5: defaultdict(list, {'cycle1': 0, 'cycle2': 7}),
              6: defaultdict(list, {'cycle1': 0, 'cycle2': 7}),
              7: defaultdict(list, {'cycle1': 0, 'cycle2': 7}),
              8: defaultdict(list, {'cycle1': 0, 'cycle2': 8}),
              9: defaultdict(list, {'cycle1': 0, 'cycle2': 9})},
             'HCC827_ctrl_Fixed': {1: defaultdict(list,
                          {'cycle1': 0, 'cycle2': 4}),
              2: defaultdict(list, {'cycle1': 0, 'cycle2': 6}),
              3: defaultdict(list, {'cycle1': 0, 'cycle2': 8}),
              4: defaultdict(list, {'cycle1': 0, 'cycle2': 10}),
              5: defaultdict(list, {'cycle1': 0, 'cycle2'

In [41]:
import pickle 

z_shift_path = data_dir / 'sun_IF_3D' / 'metadata' / 'zshift.pickle'

with open(z_shift_path, 'wb') as f:
    pickle.dump(z_shifts_all, f)

# Save Tiffile

In [43]:
from skimage import exposure, util

def contrast_str(img, n_min=0.1, n_max=100):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    return img_rescale

def make_imgs_same_dim(imgs):
    # Get max dimensions
    shapes = np.array([img.shape[1:] for img in imgs])
    min_x, min_y = shapes.min(axis=0)
    imgs_cropped = [img[:, :min_x, :min_y] for img in imgs]
    # imgs_cropped[0] = contrast_str(imgs_cropped[0])
    return imgs_cropped


In [44]:
z_shifts_all

defaultdict(dict,
            {'HCC827_O_Fixed': {1: defaultdict(list,
                          {'cycle1': 0, 'cycle2': 3}),
              2: defaultdict(list, {'cycle1': 0, 'cycle2': 5}),
              3: defaultdict(list, {'cycle1': 0, 'cycle2': 5}),
              4: defaultdict(list, {'cycle1': 0, 'cycle2': 5}),
              5: defaultdict(list, {'cycle1': 0, 'cycle2': 7}),
              6: defaultdict(list, {'cycle1': 0, 'cycle2': 7}),
              7: defaultdict(list, {'cycle1': 0, 'cycle2': 7}),
              8: defaultdict(list, {'cycle1': 0, 'cycle2': 8}),
              9: defaultdict(list, {'cycle1': 0, 'cycle2': 9})},
             'HCC827_ctrl_Fixed': {1: defaultdict(list,
                          {'cycle1': 0, 'cycle2': 4}),
              2: defaultdict(list, {'cycle1': 0, 'cycle2': 6}),
              3: defaultdict(list, {'cycle1': 0, 'cycle2': 8}),
              4: defaultdict(list, {'cycle1': 0, 'cycle2': 10}),
              5: defaultdict(list, {'cycle1': 0, 'cycle2'

In [47]:
save_path = data_dir / 'sun_IF_3D' /'imgs' / 'raw'
save_path.mkdir(parents=True, exist_ok=True)

group = df_imgs.groupby(['Cell', 'Condition', 'FOV'])
for name, df_group in group:
    
    z_shifts_cond = z_shifts_all['_'.join(name)]
        
    for i, row in tqdm(enumerate(df_group.itertuples()), total=len(df_group)):
        path = row.Path

        # Read images
        cycles = []
        imgs_all = []
        channels = []
        with h5py.File(path, "r") as f:
            for k in tqdm(natsorted(f.keys()), total=len(f.keys()), leave=False):
                cycle = k.split('_')[0]
                channel = f[k].attrs['Channel']

                imgs = f[k][:]
                if channel == 1:
                    imgs = contrast_str(imgs)
                cycles.append(cycle)
                channels.append(channel)
                imgs_all.append(imgs)
                       
        cycles = np.array(cycles)
        channels = np.array(channels)

        # Z Focus matching
        imgs_matched_z = []
        shift_min = np.min(list(z_shifts.values()))
        shift_max = np.max(list(z_shifts.values()))

        for i, img in enumerate(imgs_all):
            z = z_shifts[cycles[i]]
            imgs_matched_z.append(img[shift_max-z:])

        # Get imgs_stacked
        z_min = np.min([i.shape[0] for i in imgs_matched_z])
        imgs_stacked = np.stack([i[:z_min] for i in imgs_matched_z])

        for cycle in np.unique(cycles):
            indices = np.where(cycles == cycle)[0]
            imgs = imgs_stacked[indices,...]
            
            for z in range(imgs.shape[1]):
                 # Create temp path
                name = [row[1], row[2], cycle]
                temp_path =  save_path / '_'.join(np.array(name).astype(str))
                temp_path.mkdir(parents=True, exist_ok=True)

                file_name = '_'.join(np.array(["{:03d}".format(row[4]), 'Z'+str(z)]).astype(str)) + '.tif' # Save the z with cycle because of ashlar
                file_path = temp_path / file_name

                # if os.path.exists(file_path):
                #     continue
                
                # Write image
                tiff.imwrite(file_path, imgs[:,z,...])


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [46]:
row

Pandas(Index=0, Cell='HCC827', Condition='O', FOV='Fixed', ROI=1, Path='y:\\coskun-lab\\Thomas\\23_PLA_revision\\data\\sun_IF_3D\\hdf5\\raw\\HCC827_O_Fixed_1.hdf5')

## Copy file into new folder by matching the maximum and minium z stacks

In [ ]:
import shutil
img_dir = data_dir / 'sun_IF_3D' / 'imgs' / 'raw_per_cycle'
save_dir = data_dir /  'sun_IF_3D' / 'imgs' / 'raw_matched_z'
save_dir.mkdir(parents=True, exist_ok=True)

for condition in os.listdir(img_dir):
    print(condition)

    # Get files
    files = os.listdir(img_dir / condition)
    files = [f for f in files if 'tif' in f ]
    # Define savepath
    save_path = save_dir / condition 
    save_path.mkdir(parents=True, exist_ok=True)

    # Create temp dataframe
    df_temp = pd.DataFrame({'ROI': [f.split('_')[0] for f in files],
                        'Z': [f.split('_')[1].split('.')[0][1:] for f in files],
                        'Path': files})
    df_temp = df_temp.sort_values(by=['ROI', 'Z'], key=natsort_keygen())
    df_temp.Z = df_temp.Z.astype(np.uint8)
    min_z = df_temp.groupby(['ROI'])['Z'].max().min()
    
    # Iterate over ROI
    group = df_temp.groupby(['ROI'])
    for name, df_group in group:
        z_diff = df_group.Z.max() - min_z
        div = z_diff // 2
        rest = z_diff % 2
        df_group = df_group.iloc[div+rest:-1-div]
        df_group = df_group.reset_index(drop=True)
        for row in df_group.itertuples():
            source = img_dir / condition / row[-1]
            destination = save_path / ('_'.join([row[1], 'cycle'+str(row[0])]) + '.tif')
            # if os.path.exists(destination):
            #     continue
            shutil.copy(source, destination)


100nM_FW1_cycle1
100nM_FW1_cycle10
100nM_FW1_cycle11
100nM_FW1_cycle12
100nM_FW1_cycle13
100nM_FW1_cycle2
100nM_FW1_cycle3
100nM_FW1_cycle4
100nM_FW1_cycle5
100nM_FW1_cycle6
100nM_FW1_cycle7
100nM_FW1_cycle8
100nM_FW1_cycle9
100nM_FW2_cycle1
100nM_FW2_cycle10
100nM_FW2_cycle11
100nM_FW2_cycle12
100nM_FW2_cycle13
100nM_FW2_cycle2
100nM_FW2_cycle3
100nM_FW2_cycle4
100nM_FW2_cycle5
100nM_FW2_cycle6
100nM_FW2_cycle7
100nM_FW2_cycle8
100nM_FW2_cycle9
control_FW1_cycle1
control_FW1_cycle10
control_FW1_cycle11
control_FW1_cycle12
control_FW1_cycle13
control_FW1_cycle2
control_FW1_cycle3
control_FW1_cycle4
control_FW1_cycle5
control_FW1_cycle6
control_FW1_cycle7
control_FW1_cycle8
control_FW1_cycle9
control_FW2_cycle1
control_FW2_cycle10
control_FW2_cycle11
control_FW2_cycle12
control_FW2_cycle13
control_FW2_cycle2
control_FW2_cycle3
control_FW2_cycle4
control_FW2_cycle5
control_FW2_cycle6
control_FW2_cycle7
control_FW2_cycle8
control_FW2_cycle9


# Registration 

In [21]:
from ashlar import fileseries, thumbnail,reg
import matplotlib.pyplot as plt
from ashlar.scripts.ashlar import process_axis_flip

In [22]:
# Loop all images
imgs_dir = data_dir / 'sun_IF_v2' /'imgs' / 'raw'
save_dir = data_dir / 'sun_IF_v2' /'imgs' / 'registered'
save_dir.mkdir(parents=True, exist_ok=True)

imgs_dir_list = os.listdir(imgs_dir)

for dir_path in tqdm(imgs_dir_list):
    
    # Create reader for each cycle
    readers = []
    for i in range(1, 3):
        reader = fileseries.FileSeriesReader(
            str(imgs_dir / dir_path),
            pattern='{series}_'+f'cycle{i}.tif',
            overlap=0.29,
            width=1,
            height=1,
            layout='snake',
            direction='horizontal',
            pixel_size=0.18872, 
        )
        readers.append(reader)
    reader_1 = readers[0]
    
    # Run stitching
    aligner0 = reg.EdgeAligner(reader_1, channel=0, filter_sigma=2, verbose=False,)
    aligner0.run()
    
    # Generate merge image for 1 cycle
    # Parramter
    mosaic_args = {}
    mosaic_args['verbose'] = False

    mosaic = reg.Mosaic(
            aligner0,aligner0.mosaic_shape,**mosaic_args
        )
    writer_class = reg.TiffListWriter
    writer = writer_class(
            [mosaic], str(save_dir / (dir_path + '_cycle1_ch{channel}.ome.tif'))
    )
    writer.run()
    
    # Loop through rest of cycles
    aligners = list()
    aligners.append(aligner0)

    for j in range(1, len(readers)):
        aligners.append(
            reg.LayerAligner(readers[j], aligners[0], channel=0, filter_sigma=2, verbose=False)
        )
        aligners[j].run()
        mosaic = reg.Mosaic(
            aligners[j], aligners[0].mosaic_shape,**mosaic_args
        )
        writer = writer_class(
                [mosaic], str(save_dir / (dir_path +'_cycle'+str(j+1)+'_ch{channel}.ome.tif'))
        )
        writer.run()
    

  0%|          | 0/4 [00:00<?, ?it/s]

    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [1. 6.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [-1. -3.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [ 4. 11.]
    assembling thumbnail 1/1
    assembling thumbnail 1/1
    estimated cycle offset [y x] = [ 6. -3.]


# Save data

In [23]:
def get_info(data_raw, marker_dict):
    cells = []
    conditions = []
    fovs = []
    cycles = []
    channels = []
    markers = []
    paths = [] 

    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(data_raw):
        for name in sorted(filenames):
            if "tif" in name:
                # Get information from image name
                n_split = name.split('_')
                cell=n_split[0]                       
                cond=n_split[1]
                fov=n_split[2]
                cycle=n_split[3]
                ch = n_split[4][:3]
                try:
                    marker = marker_dict[cycle][ch]
                except:
                    continue 
                
                cells.append(cell)
                conditions.append(cond)
                fovs.append(fov)
                cycles.append(cycle)
                channels.append(ch)
                markers.append(marker)
                paths.append(os.path.join(dirpath, name))
                
    info = {
            'Cell': cells,
            "Condition": conditions,
            "FOV": fovs,
            "Cycle": cycles,
            "Channels": channels,
            "Markers": markers,
            "Path": paths
        }

    df = pd.DataFrame(info)
    return df

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

def read_img(path):
    return skimage.io.imread(path, as_gray=True)

def get_min(imgs):
    shapes = np.array([np.array(img.shape) for img in imgs])
    return np.min(shapes, axis=0)

In [24]:
from collections import defaultdict

markers_map_new = defaultdict(dict)
for k,v in markers_map.items():
    for i, (ch,marker) in enumerate(v.items()):
        markers_map_new[k][f'ch{i}'] = marker
        

In [25]:
data_raw = data_dir  / 'sun_IF_v2' /'imgs' / 'registered'
df_meta_path = data_dir /  'sun_IF_v2' / 'metadata' / 'info_sti.csv'

try:
    df_meta_path.parent.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_meta_path.is_file()

if not df_exist:
    print('Created df')
    df = get_info(data_raw, markers_map_new)
    df.to_csv(df_meta_path, index=False)
else:
    print('Loaded df')
    df = pd.read_csv(df_meta_path)

Folder is already there
Created df


In [26]:
df

,Cell,Condition,FOV,Cycle,Channels,Markers,Path
0,HCC827,O,Fixed,cycle1,ch0,Hoechst,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
1,HCC827,O,Fixed,cycle1,ch1,DR5,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
2,HCC827,O,Fixed,cycle1,ch2,TRAIL,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
3,HCC827,O,Fixed,cycle2,ch0,Hoechst,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
4,HCC827,O,Fixed,cycle2,ch1,Phalloidin,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
5,HCC827,ctrl,Fixed,cycle1,ch0,Hoechst,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
6,HCC827,ctrl,Fixed,cycle1,ch1,DR5,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
7,HCC827,ctrl,Fixed,cycle1,ch2,TRAIL,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
8,HCC827,ctrl,Fixed,cycle2,ch0,Hoechst,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
9,HCC827,ctrl,Fixed,cycle2,ch1,Phalloidin,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...


In [27]:
df_imgs_path = data_dir / 'sun_IF_v2'  /'metadata' / 'imgs_reg.csv'

temp_path =data_dir /  'sun_IF_v2'  /'hdf5' / 'registered'
try:
    temp_path.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Folder is already there")

df_exist = df_imgs_path.is_file()

if not df_exist:
    print('Created df')
    group = df.groupby(['Cell', 'Condition','FOV'])
    rows = []

    for name, df_group in tqdm(group, total=len(group)):
        file_name = '_'.join(np.array(name).astype(str)) + '.hdf5'
        file_path = temp_path / file_name
        rows.append(list(name)+[file_path])
        
        # if file_path.exists():
        #     continue
        
        channels = df_group.Channels.to_list()
        cycles = df_group.Cycle.to_list()
        markers = df_group.Markers.to_list()
        paths = df_group.Path.to_numpy()
            
        imgs = joblib_loop(read_img, paths)
        min_shape = get_min(imgs)
        imgs_cropped = np.array([img[:min_shape[0], :min_shape[1]] for img in imgs])
        info = {"Cycle": cycles, "Channel": channels, "Marker": markers}
            
            # hdf5 as Channel -> Z mapping
        save_hdf5(file_path, 'imgs', imgs_cropped, info)
    df_imgs = pd.DataFrame(rows, columns=['Cell', 'Condition', 'FOV', 'Path'])        
    df_imgs.to_csv(df_imgs_path, index=False)
else:
    print('Loaded df')
    df_imgs = pd.read_csv(df_imgs_path)

Created df


  0%|          | 0/4 [00:00<?, ?it/s]

In [28]:
df_imgs

,Cell,Condition,FOV,Path
0,HCC827,O,Fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
1,HCC827,ctrl,Fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
2,PC9,O,Fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
3,PC9,ctrl,Fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...


# Segmentation 

In [30]:
from skimage.exposure import match_histograms, rescale_intensity
import cv2
from skimage import img_as_ubyte
from skimage import exposure, util, filters, restoration

def random_crop(image, NEW_IMG_HEIGHT, NEW_IMG_WIDTH):
    cropped_image = tf.image.random_crop(
      image, size=[3, NEW_IMG_HEIGHT, NEW_IMG_WIDTH])

    return cropped_image

def make_color_overlay(input_data):
    """Create a color overlay from 2 channel image data
    
    Args:
        input_data: stack of input images
    
    Returns:
        numpy.array: color-adjusted stack of overlays in RGB mode
    """
    RGB_data = np.zeros(input_data.shape[:3] + (3, ), dtype='float32')
    
    # rescale channels to aid plotting
    for img in range(input_data.shape[0]):
        for channel in range(input_data.shape[-1]):
            # get histogram for non-zero pixels
            percentiles = np.percentile(input_data[img, :, :, channel][input_data[img, :, :, channel] > 0],
                                            [0, 100])
            rescaled_intensity = rescale_intensity(input_data[img, :, :, channel],
                                                       in_range=(percentiles[0], percentiles[1]),
                                                       out_range='float32')
            RGB_data[img, :, :, channel + 1] = rescaled_intensity
        
    # create a blank array for red channel
    return RGB_data

def contrast_str(img, n_min=10, n_max=100):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = rescale_intensity(img, in_range=(p2, p98))
    img_rescale = util.img_as_ubyte(img_rescale)
    return img_rescale

In [31]:
df_imgs_path = data_dir /  'sun_IF_v2'  /'metadata' / 'imgs_reg.csv'
df_imgs = pd.read_csv(df_imgs_path)

In [32]:
df_imgs

,Cell,Condition,FOV,Path
0,HCC827,O,Fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
1,HCC827,ctrl,Fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
2,PC9,O,Fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...
3,PC9,ctrl,Fixed,y:\coskun-lab\Thomas\23_PLA_revision\data\sun_...


In [ ]:
for row in tqdm(df_imgs.iloc[5:6].itertuples(), total=len(df_imgs)):
    path = row.Path
    
    # Read images
    with h5py.File(path, "r") as f:
        
        markers = f['imgs'].attrs['Marker']
        imgs = f['imgs'][:]


  0%|          | 0/8 [00:00<?, ?it/s]

In [19]:
import napari 

napari.view_image(np.stack(imgs), name=markers, channel_axis=0, contrast_limits=[0,1])

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1722.5, 2300.5), zoom=0.16735284875790893, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 3446.0, 1.0), (0.0, 4602.0, 1.0)), current_step=(1723, 2301), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Hoechst' at 0x23b3a598ac0>, <Image layer 'DR5' at 0x23b3927e4d0>, <Image layer 'TRAIL' at 0x23b3a655360>, <Image layer 'Hoechst [1]' at 0x23b3b5eb070>, <Image layer 'Phalloidin' at 0x23b3b68c820>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None)

In [33]:
# Define markers to use for cytosolic segmentation
cyto_markers = ['Phalloidin']

# Define folder and create if don't exsit
whole_seg_path = data_dir /  'sun_IF_v2'  / 'imgs' / 'segmentation'
whole_seg_path.mkdir(parents=True, exist_ok=True)

# Save combined images
for row in df_imgs.itertuples():
    # Read image
    path = row.Path
    with h5py.File(path, "r") as f:
        imgs = f['imgs'][:]
        markers = f['imgs'].attrs['Marker']

    # Get dapi and cyto imgaes
    indices = np.isin(markers, cyto_markers)
    img_dapi =  imgs[0]
    imgs_cyto = imgs[indices,:]
    
    # Contrast streching and combine to rgb image
    img_dapi = contrast_str(img_dapi, n_max=99.9)
        
    imgs_cyto_scaled = []
    for img in imgs_cyto:
        imgs_cyto_scaled = [contrast_str(img, n_max=99.9)]
    img_cyto = np.max(np.array(imgs_cyto_scaled), axis=0)
    img_rgb = np.stack([np.zeros(img_dapi.shape),img_cyto, img_dapi], axis=0).astype(np.uint8)
    
    # Crop and save
    file_name = f'{"_".join(row[1:4])}.tif'
    file_path = whole_seg_path / file_name
    tiff.imwrite(file_path, img_rgb)

